In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.ESIM.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5435.88it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:06<00:00, 3072.26it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 1419306.07it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 4881.96it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:06<00:00, 3091.21it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 617961.23it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7718.72it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fa7f25bea20>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fa86404fe80>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10,
    batch_size=20,
    resample=True,
    sort=False,
    shuffle=True,
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20,
    sort=False,
    shuffle=False
)

In [9]:
padding_callback = mz.models.ESIM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.ESIM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['mask_value'] = 0
model.params['dropout'] = 0.2
model.params['hidden_size'] = 200
model.params['lstm_layer'] = 1

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ESIM(
  (embedding): Embedding(30058, 300, padding_idx=0)
  (rnn_dropout): RNNDropout(p=0.2, inplace=False)
  (input_encoding): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 100, bidirectional=True)
    )
  )
  (attention): BidirectionalAttention()
  (projection): Sequential(
    (0): Linear(in_features=800, out_features=200, bias=True)
    (1): ReLU()
  )
  (composition): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(200, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=800, out_features=200, bias=True)
    (2): Tanh()
    (3): Dropout(p=0.2, inplace=False)
  )
  (out): Linear(in_features=200, out_features=1, bias=True)
)
Trainable params:  9901201


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

[Iter-255 Loss-1.899]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5767 - normalized_discounted_cumulative_gain@5(0.0): 0.6467 - mean_average_precision(0.0): 0.5947



[Iter-510 Loss-0.546]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5611 - normalized_discounted_cumulative_gain@5(0.0): 0.6228 - mean_average_precision(0.0): 0.5789



[Iter-765 Loss-0.115]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5135 - normalized_discounted_cumulative_gain@5(0.0): 0.5737 - mean_average_precision(0.0): 0.5287



[Iter-1020 Loss-0.054]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5062 - normalized_discounted_cumulative_gain@5(0.0): 0.5619 - mean_average_precision(0.0): 0.5209



[Iter-1275 Loss-0.035]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5519 - normalized_discounted_cumulative_gain@5(0.0): 0.6121 - mean_average_precision(0.0): 0.5682



[Iter-1530 Loss-0.033]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5305 - normalized_discounted_cumulative_gain@5(0.0): 0.5881 - mean_average_precision(0.0): 0.5413



[Iter-1785 Loss-0.031]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5362 - normalized_discounted_cumulative_gain@5(0.0): 0.6048 - mean_average_precision(0.0): 0.5597



[Iter-2040 Loss-0.022]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.551 - normalized_discounted_cumulative_gain@5(0.0): 0.6143 - mean_average_precision(0.0): 0.5733



[Iter-2295 Loss-0.018]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.561 - normalized_discounted_cumulative_gain@5(0.0): 0.6109 - mean_average_precision(0.0): 0.5726

